# FE019 - Reduce Size

In [1]:
import pandas as pd

In [3]:
gf = pd.read_csv('../fi/Good_Features_By_Type.csv')
gf_2jhc = gf.loc[gf['2JHC'] == True]['feature'].tolist()
gf_2jhc = [x for x in gf_2jhc if 'meta_fc' not in x]

In [ ]:
tr_2jhc = pd.read_parquet('../data/FE019/FE019-train-2JHC.parquet')
# 2JHC Good Features Plus New Features
tr_2jhc[gf_2jhc + NEW_FEATURES + ['scalar_coupling_constant']].to_parquet('../data/FE019/FE019-train-2JHC-reduced.parquet')

In [4]:
test_2jhc = pd.read_parquet('../data/FE019/FE019-test-2JHC.parquet')
test_2jhc[gf_2jhc + NEW_FEATURES].to_parquet('../data/FE019/FE019-test-2JHC-reduced.parquet')
test_2jhc = pd.read_parquet('../data/FE019/FE019-test-2JHC.parquet')

## 1JHC

In [3]:
gf = pd.read_csv('../fi/Good_Features_By_Type.csv')
gf_1jhc = gf.loc[gf['1JHC'] == True]['feature'].tolist()
gf_1jhc = [x for x in gf_1jhc if 'meta_fc' not in x]

In [4]:
# 1JHC Good Features Plus New Features
tr_1jhc = pd.read_parquet('../data/FE019/FE019-train-1JHC.parquet')
tr_1jhc[gf_1jhc + NEW_FEATURES + ['scalar_coupling_constant']].to_parquet('../data/FE019/FE019-train-1JHC-reduced.parquet')

In [4]:
test_1jhc = pd.read_parquet('../data/FE019/FE019-test-1JHC.parquet')
test_1jhc[gf_1jhc + NEW_FEATURES].to_parquet('../data/FE019/FE019-test-1JHC-reduced.parquet')

In [2]:
NEW_FEATURES = ['dist_C_0_x',
                 'dist_C_1_x',
                 'dist_C_2_x',
                 'dist_C_3_x',
                 'dist_C_4_x',
                 'dist_F_0_x',
                 'dist_F_1_x',
                 'dist_F_2_x',
                 'dist_F_3_x',
                 'dist_F_4_x',
                 'dist_H_0_x',
                 'dist_H_1_x',
                 'dist_H_2_x',
                 'dist_H_3_x',
                 'dist_H_4_x',
                 'dist_N_0_x',
                 'dist_N_1_x',
                 'dist_N_2_x',
                 'dist_N_3_x',
                 'dist_N_4_x',
                 'dist_O_0_x',
                 'dist_O_1_x',
                 'dist_O_2_x',
                 'dist_O_3_x',
                 'dist_O_4_x',
                 'dist_C_0_y',
                 'dist_C_1_y',
                 'dist_C_2_y',
                 'dist_C_3_y',
                 'dist_C_4_y',
                 'dist_F_0_y',
                 'dist_F_1_y',
                 'dist_F_2_y',
                 'dist_F_3_y',
                 'dist_F_4_y',
                 'dist_H_0_y',
                 'dist_H_1_y',
                 'dist_H_2_y',
                 'dist_H_3_y',
                 'dist_H_4_y',
                 'dist_N_0_y',
                 'dist_N_1_y',
                 'dist_N_2_y',
                 'dist_N_3_y',
                 'dist_N_4_y',
                 'dist_O_0_y',
                 'dist_O_1_y',
                 'dist_O_2_y',
                 'dist_O_3_y',
                 'dist_O_4_y',
                 'dist',
                 'dist_inv2',
                 'distance_closest_0',
                 'distance_closest_1',
                 'distance_farthest_0',
                 'distance_farthest_1',
                 'molecule_atom_index_0_dist_mean',
                 'molecule_atom_index_0_dist_mean_diff',
                 'molecule_atom_index_0_dist_min',
                 'molecule_atom_index_0_dist_min_diff',
                 'molecule_atom_index_0_dist_std',
                 'molecule_atom_index_1_dist_mean',
                 'molecule_atom_index_1_dist_mean_diff',
                 'molecule_atom_index_1_dist_min',
                 'molecule_atom_index_1_dist_min_diff',
                 'molecule_atom_index_1_dist_std',
                 'molecule_type_dist_mean',
                 'molecule_type_dist_mean_diff',
                 'rc_A',
                 'rc_B',
                 'rc_C',
                 'mu',
                 'alpha',
                 'homo',
                 'lumo',
                 'gap',
                 'zpve',
                 'Cv',
                 'freqs_min',
                 'freqs_max',
                 'freqs_mean',
                 'mulliken_min',
                 'mulliken_max',
                 'mulliken_atom_0',
                 'mulliken_atom_1']